# Introduction to SQL with Python

## Functions and types

In [ ]:
# !pip install --user ipython-sql

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite://

In [ ]:
%%sql -- Re-create the database / table / rows
CREATE TABLE Writer (FirstName, LastName, YearOfBirth, YearOfDeath);
INSERT INTO Writer VALUES
    ('William', 'Shakespeare', null, 1516),
    ('Bertold', 'Brecht', 1898, 1956),
    ('Ernest', 'Hemingway', 1899, 1961),
    ('Oliver', 'Sacks', 1933, 2015),
    ('Richard', 'Bird', 1943, null),
    ('Hans Petter', 'Langtangen', 1962, null),
    ('Jan Jacob', 'Slauerhoff', 1898, 1936),
    ('William', 'Burroughs', 1914, 1997),
    ('Ira', 'Kalet', 1944, null);


### Expressions

Just like in Python, SQL allows us to make complex expressions from simpler building blocks. For example, the usual arithmetic operators such as `+`, `-`, `*` and `/` are supported:

In [ ]:
%%sql SELECT FirstName, LastName, (YearOfDeath - YearOfBirth)
FROM Writer
WHERE LastName LIKE 'S%';

The above goes wrong for writers that are still alive, but you get the idea. The parentheses here are not strictly required, but help visually group the different parts nicely.

Often the syntax of SQL expressions will be similar to what you've learned for Python, but one notable exception is concatenation of strings. In SQL this is done with two _pipe_ characters,
like so:

In [ ]:
%%sql
SELECT FirstName || ' ' || LastName AS FullName
FROM Writer
WHERE LastName LIKE 'S%';

Notice that we used the <code>AS</code> keyword to give a name to the **full_name** column, which did not have a name because we effectively made it out of thin air here. Without such an alias, as in the previous slide, the result set will just use the expression itself as column header.

### Functions

We can further composite expressions using functions. Unfortunately, this is one aspect in which different implementations of SQL database don't agree on syntax and exactly which functions to support... So you should probably keep a cheat sheet at hand for your DB when really digging in.

Functions that you will definitely find useful include <code>COUNT</code>, <code>MIN</code>, <code>MAX</code>, <code>AVG</code>, and <code>DISTINCT</code>:

In [ ]:
%%sql SELECT COUNT(*) AS num_writers
FROM Writer;

In [ ]:
%%sql SELECT MIN(YearOfBirth), MAX(YearOfBirth), AVG(YearOfBirth)
FROM Writer;

The last column above looks a bit strange, because we're not used to years with fractional parts. This gets us wondering about types, and we'll get back to that shortly.

In [ ]:
%%sql SELECT DISTINCT(FirstName)
FROM Writer;

We can of course also nest function calls, like so:

In [ ]:
%%sql SELECT COUNT(DISTINCT(FirstName)) AS NumFirstNames
FROM Writer;

To conclude this section, let's throw together some commonly used functions in one query:

In [ ]:
%%sql SELECT
    LENGTH(FirstName) AS length,
    SUBSTR(FirstName, 1, 1) AS initial,
    UPPER(LastName) AS upper,
    IFNULL(YearOfDeath, 2017) AS year
FROM Writer;

It is slightly morbid, but substituting <code>2017</code> for the rows without year of death -- which is what <code>IFNULL</code> does above -- allows us to compute the **age** attribute like before, even for undeceased authors. For example, let's find our 5 eldest writers:

In [ ]:
%%sql 
SELECT 
    FirstName, 
    LastName, 
    (IFNULL(YearOfDeath, 2017) - YearOfBirth) AS age
FROM Writer
ORDER BY age DESC
LIMIT 5;

Thusfar we've glossed over one important difference between arithmetic expressions and functions like <code>LENGTH</code> and <code>SUBSTR</code> on the one hand, which work horizontally per row (or on a single cell), and on the other hand functions like <code>COUNT</code>, <code>MIN</code> and <code>MAX</code>, which work vertically per column.

Functions of the vertical persuasion can be applied to the column as a whole, i.e. all the rows in the table, as was done above. But they can also be made to target _groups_ of rows. We'll introduce one more useful function, <code>SUM</code> to demonstrate this:

In [ ]:
%%sql SELECT LENGTH(FirstName) AS len, COUNT(*), SUM(YearOfDeath)
FROM Writer
GROUP BY len;

Note that we've used an alias **len** for the first column of the result set, and that we were able to refer back to it using that alias to write our <code>GROUP BY</code> clause.

It doesn't really tell us anything interesting, but what happened here is we've grouped together writers having the same number of characters in their first name. For each of these groups, we've counted how many writers it contains and summed together their years of death.

This kind of grouping can be quite a powerful weapon, especially in queries where we glue together rows from several different tables using <code>JOIN</code>s. More about that later.

### Types

When we created the <code>writer</code> table, we did not specify for each column what kind of data we expect it to hold. Usually we would do that, and in fact most database implementations demand it, but in our case the database just made an educated guess depending on what we insert.

So let's try that again, but this time we're going to be more explicit about types:

In [ ]:
%sql DROP TABLE Writer;

In [ ]:
%%sql 
CREATE TABLE Writer (
    FirstName TEXT, 
    LastName TEXT, 
    YearOfBirth INTEGER, 
    YearOfDeath INTEGER
);

In [ ]:
%%sql -- Re-populate the table
INSERT INTO Writer VALUES
    ('William', 'Shakespeare', null, 1516),
    ('Bertold', 'Brecht', 1898, 1956),
    ('Ernest', 'Hemingway', 1899, 1961),
    ('Oliver', 'Sacks', 1933, 2015),
    ('Richard', 'Bird', 1943, null),
    ('Hans Petter', 'Langtangen', 1962, null),
    ('Jan Jacob', 'Slauerhoff', 1898, 1936),
    ('William', 'Burroughs', 1914, 1997),
    ('Ira', 'Kalet', 1944, null);


Types are another aspect in which SQL implementations tend to have significant differences. For example, some would have us specify the possible lengths of <code>TEXT</code> fields, so that it can take that into account when allocating memory or diskspace. Others, like sqlite which we're using here, are more lenient in this regard.

Generally though, there will be distinct types to contain textual data and numerical data. As in Python datatypes, it is sometimes important to distinguish numerical types further into integer ("whole") numbers and real ("floating point") numbers. We've already seen an aspect of this briefly when we were averaging years earlier.

Often there will also be more specialized types, for instance for temporal data (to capture moments in time) or spatial data (such as latitude/longitude coordinates).

Looking back at some of the operators and functions we've been using, it becomes apparent that some of them, like the <code>LENGTH</code> function and the concatenation operator <code>||</code>, expect their input to be strings. Whereas others, such as <code>AVG</code> and the various arithmetic operators, expect their operands to be numbers.

Fortunately we can usually count on the database doing the "right thing" when we use a function on an unexpected data type, by coercing it to the right type.


We can also classify functions in terms of the types of value that they yield (return): <code>LENGTH</code> gives us back a number, but <code>SUBSTR</code> returns a string. Specialized types usually come with their own catalogues of functions, for example to deal with particulars of calendars (leap years and the like) or geodetic distance.

### Introducing the Chinook DB

In [ ]:
# Connect to the Chinook database
%sql sqlite:///Chinook.sqlite

In [ ]:
%%sql

-- Show some rows from Artist table
SELECT *
FROM Artist
LIMIT 5;

In [ ]:
%%sql

-- Show some rows from Album table
SELECT *
FROM Album
LIMIT 5;

In [ ]:
%%sql

-- Show some rows from Genre table
SELECT *
FROM Genre
LIMIT 5;

In [ ]:
%%sql

-- Show some rows from Track table
SELECT *
FROM Track
LIMIT 5;